# Running microkinetic simulations with CARE

This notebook tutorial provides a basic example on how to run microkinetic modeling (MKM) simulations of CRNs built with CARE.
We provide two ways to run simulations: with solvers implemented with Scipy, or with Julia solvers.

IN PROGRESS

## Load CRN

## MKM with Scipy

## MKM with Julia

### CPU

### GPU

You need a GPU with CUDA to run the next part of the code.